# Types in Julia

In this notebook we will start exploring types by implementing a type to implement algorithmic differentiation.

### What is a type?

A *type* can be thought of as a label that is associated with data stored in memory; this label tells Julia how to interpret the data. For example:

In [1]:
x = 1
y = 1.3

1.3

In [2]:
sizeof(x), sizeof(y)

(8, 8)

In [3]:
typeof(x), typeof(y)

(Int64, Float64)

In [5]:
z = reinterpret(Float64, x)

5.0e-324

In [4]:
bitstring(x)

"0000000000000000000000000000000000000000000000000000000000000001"

In [6]:
bitstring(z)

"0000000000000000000000000000000000000000000000000000000000000001"

Both variables are stored in 8 bytes (64 bits), but one is interpreted as an integer and the other as a floating-point number.  Similarly, a pair of two numbers may be intepreted as a complex number, or an interval, or a dual number, or...; although the same information may be stored (two numbers), we want each of these different *kinds* or *types* of objects to be treated differently. 

## Algorithmic differentiation

In the previous notebook we used `ForwardDiff.jl` to automatically differentiate a function. Here we will see how to implement a simple version of this.

The idea is to approximate a (nice enough) function $f$ near a point $a$ by a Taylor series of order 1, i.e. a straight line passing through $(a, f(a))$, with slope equal to the derivative $f'(a)$:
    
$f(x) \simeq f(a) + \epsilon f'(a)$,

where $\epsilon := x - a$.

We now use this to derive the standard rules for the derivative of a sum and product:

$$f(x) + g(x) \simeq [f(a) + g(a)] + \epsilon [f'(a) + g'(a)]$$

$$f(x) \cdot g(x) \simeq [f(a) \cdot g(a)] + \epsilon [f(a) g'(a) + g(a) f'(a)],$$

where we suppose that $\epsilon$ is small enough that $\epsilon^2 = 0$, or alternatively just "take the linear part".

### Defining a composite type

We see that by using just two pieces of information, namely the value $f(a)$ and the derivative $f'(a)$, we can represent a function $f$ near a given point $a$. 

The pair $(f(a), f'(a))$ is often called a **dual number**. We see that is has certain **behaviours** under arithmetic operations. Whenever we have a new behaviour, a *new type is lurking*!

We group the two values into a **composite type**. We can think of a composite type as specifying the structure of a box containing several pieces of information (data) inside. Defining a composite type with two **fields** (pieces of information) has the following syntax:

In [7]:
struct MyType
    a
    b::Int
end

Here we have additionally specified that the information stored in the field `b` must be of type `Int` using the **type annotation operator**, `::`.

Creating an object of that type is accomplished as follows:

In [8]:
x = MyType(3, 4)

MyType(3, 4)

We can extract information as follows:

In [9]:
x.a

3

#### Exercise 1

1. Define a composite type `Dual` with fields `value` and `deriv` of type `Float64`.


2. Create two `Dual` numbers `x` and `y`.


3. What happens if you try to add `x` and `y` together?


4. Make a function `add` that adds `x` and `y` and returns a new `Dual` number, following the rules we found above.

In [10]:
struct Dual
    value::Float64
    deriv::Float64
end

In [11]:
x = Dual(3, 4)

Dual(3.0, 4.0)

In [12]:
methods(Dual)

# 2 methods for generic function "(::Type)":
[1] Dual(value::Float64, deriv::Float64) in Main at In[10]:2
[2] Dual(value, deriv) in Main at In[10]:2

In [13]:
convert(Float64, 3)

3.0

In [14]:
convert(Float64, "hello")

MethodError: MethodError: Cannot `convert` an object of type String to an object of type Float64
Closest candidates are:
  convert(::Type{T<:Number}, !Matched::T<:Number) where T<:Number at number.jl:6
  convert(::Type{T<:Number}, !Matched::Number) where T<:Number at number.jl:7
  convert(::Type{T<:Number}, !Matched::Base.TwicePrecision) where T<:Number at twiceprecision.jl:250
  ...

In [15]:
y = Dual(5, 6)

Dual(5.0, 6.0)

In [16]:
x + y

MethodError: MethodError: no method matching +(::Dual, ::Dual)
Closest candidates are:
  +(::Any, ::Any, !Matched::Any, !Matched::Any...) at operators.jl:502

In [17]:
add(x::Dual, y::Dual) = Dual(x.value + y.value, x.deriv + y.deriv)

add (generic function with 1 method)

In [18]:
add(x, y)

Dual(8.0, 10.0)

In [19]:
+(x::Dual, y::Dual) = Dual(x.value + y.value, x.deriv + y.deriv)

ErrorException: error in method definition: function Base.+ must be explicitly imported to be extended

## Implementing arithmetic for a type

We would like to be able to use `+` and `*` for our new `Dual` type, rather than typing `add(x, y)`. To do so, we need to do the following

In [20]:
import Base: +, *

In [21]:
+(x::Dual, y::Dual) = Dual(x.value + y.value, x.deriv + y.deriv)

+ (generic function with 164 methods)

In [22]:
Base.+(x::Dual, y::Dual) = Dual(x.value + y.value, x.deriv + y.deriv)

ErrorException: syntax: invalid function name ".+"

In [23]:
Base.:+(x::Dual, y::Dual) = Dual(x.value + y.value, x.deriv + y.deriv)

In [ ]:
+(x::Dual, y::Dual) = add(x, y)

In [32]:
function +(x::Dual, y::Dual)
    return add(x, y)
end

+ (generic function with 164 methods)

In Julia, `+` and `*` are just functions. They are defined in `Base` (a module containing basic function definitions) and must be `import`ed before being **extended**. They consist of many different **methods** (versions):

In [26]:
@which +(3)

+(x::Number) in Base at operators.jl:477

In [27]:
-(3)

-3

In [28]:
methods(+)

# 164 methods for generic function "+":
[1] +(x::Bool, z::Complex{Bool}) in Base at complex.jl:277
[2] +(x::Bool, y::Bool) in Base at bool.jl:104
[3] +(x::Bool) in Base at bool.jl:101
[4] +(x::Bool, y::T) where T<:AbstractFloat in Base at bool.jl:112
[5] +(x::Bool, z::Complex) in Base at complex.jl:284
[6] +(a::Float16, b::Float16) in Base at float.jl:392
[7] +(x::Float32, y::Float32) in Base at float.jl:394
[8] +(x::Float64, y::Float64) in Base at float.jl:395
[9] +(z::Complex{Bool}, x::Bool) in Base at complex.jl:278
[10] +(z::Complex{Bool}, x::Real) in Base at complex.jl:292
[11] +(::Missing, ::Missing) in Base at missing.jl:96
[12] +(::Missing) in Base at missing.jl:83
[13] +(::Missing, ::Number) in Base at missing.jl:97
[14] +(level::Base.CoreLogging.LogLevel, inc::Integer) in Base.CoreLogging at logging.jl:106
[15] +(c::BigInt, x::BigFloat) in Base.MPFR at mpfr.jl:406
[16] +(a::BigInt, b::BigInt, c::BigInt, d::BigInt, e::BigInt) in Base.GMP at gmp.jl:434
[17] +(a::BigInt, b::BigInt, c::BigInt, d::BigInt) in Base.GMP at gmp.jl:433
[18] +(a::BigInt, b::BigInt, c::BigInt) in Base.GMP at gmp.jl:432
[19] +(x::BigInt, y::BigInt) in Base.GMP at gmp.jl:403
[20] +(x::BigInt, c::Union{UInt16, UInt32, UInt64, UInt8}) in Base.GMP at gmp.jl:440
[21] +(x::BigInt, c::Union{Int16, Int32, Int64, Int8}) in Base.GMP at gmp.jl:446
[22] +(a::BigFloat, b::BigFloat, c::BigFloat, d::BigFloat, e::BigFloat) in Base.MPFR at mpfr.jl:556
[23] +(a::BigFloat, b::BigFloat, c::BigFloat, d::BigFloat) in Base.MPFR at mpfr.jl:549
[24] +(a::BigFloat, b::BigFloat, c::BigFloat) in Base.MPFR at mpfr.jl:543
[25] +(x::BigFloat, c::BigInt) in Base.MPFR at mpfr.jl:402
[26] +(x::BigFloat, y::BigFloat) in Base.MPFR at mpfr.jl:371
[27] +(x::BigFloat, c::Union{UInt16, UInt32, UInt64, UInt8}) in Base.MPFR at mpfr.jl:378
[28] +(x::BigFloat, c::Union{Int16, Int32, Int64, Int8}) in Base.MPFR at mpfr.jl:386
[29] +(x::BigFloat, c::Union{Float16, Float32, Float64}) in Base.MPFR at mpfr.jl:394
[30] +(x::Dates.CompoundPeriod, y::Dates.CompoundPeriod) in Dates at /Users/osx/buildbot/slave/package_osx64/build/usr/share/julia/stdlib/v1.1/Dates/src/periods.jl:343
[31] +(x::Dates.CompoundPeriod, y::Dates.Period) in Dates at /Users/osx/buildbot/slave/package_osx64/build/usr/share/julia/stdlib/v1.1/Dates/src/periods.jl:341
[32] +(x::Dates.CompoundPeriod, y::Dates.TimeType) in Dates at /Users/osx/buildbot/slave/package_osx64/build/usr/share/julia/stdlib/v1.1/Dates/src/periods.jl:371
[33] +(x::Dates.Date, y::Dates.Day) in Dates at /Users/osx/buildbot/slave/package_osx64/build/usr/share/julia/stdlib/v1.1/Dates/src/arithmetic.jl:74
[34] +(x::Dates.Date, y::Dates.Week) in Dates at /Users/osx/buildbot/slave/package_osx64/build/usr/share/julia/stdlib/v1.1/Dates/src/arithmetic.jl:72
[35] +(dt::Dates.Date, z::Dates.Month) in Dates at /Users/osx/buildbot/slave/package_osx64/build/usr/share/julia/stdlib/v1.1/Dates/src/arithmetic.jl:55
[36] +(dt::Dates.Date, y::Dates.Year) in Dates at /Users/osx/buildbot/slave/package_osx64/build/usr/share/julia/stdlib/v1.1/Dates/src/arithmetic.jl:28
[37] +(dt::Dates.Date, t::Dates.Time) in Dates at /Users/osx/buildbot/slave/package_osx64/build/usr/share/julia/stdlib/v1.1/Dates/src/arithmetic.jl:19
[38] +(t::Dates.Time, dt::Dates.Date) in Dates at /Users/osx/buildbot/slave/package_osx64/build/usr/share/julia/stdlib/v1.1/Dates/src/arithmetic.jl:20
[39] +(x::Dates.Time, y::Dates.TimePeriod) in Dates at /Users/osx/buildbot/slave/package_osx64/build/usr/share/julia/stdlib/v1.1/Dates/src/arithmetic.jl:78
[40] +(dt::Dates.DateTime, z::Dates.Month) in Dates at /Users/osx/buildbot/slave/package_osx64/build/usr/share/julia/stdlib/v1.1/Dates/src/arithmetic.jl:48
[41] +(dt::Dates.DateTime, y::Dates.Year) in Dates at /Users/osx/buildbot/slave/package_osx64/build/usr/share/julia/stdlib/v1.1/Dates/src/arithmetic.jl:24
[42] +(x::Dates.DateTime, y::Dates.Period) in Dates at /Users/osx/buildbot/slave/package_osx64/build/usr/share/julia/stdlib/v1.1/Dates/src/arithmetic.jl:76
[43]

In [29]:
x

Dual(3.0, 4.0)

In [30]:
y

Dual(5.0, 6.0)

In [31]:
x + y

Dual(8.0, 10.0)

We can add more methods that work on our own types. (We are not allowed to modify their behaviour on combinations of types that to not contain our user-defined types; doing so is known as "type piracy" and can affect other people's code in unexpected ways.)

#### Exercise 2

1. Import the `+` and `*` functions from `Base` and implement them for the `Dual` type.
They should return a new `Dual` object.


2. Check that the number of methods has changed. 


3. Use `@which x + y` to check that Julia knows which method to use when adding two `Dual`s.


4. Can you define `x + a` for a `Dual` number `x` and a real number `a`? What happens 

In [34]:
x::Dual * y::Dual = Dual(x.value * y.value,
                         x.value * y.deriv + x.deriv * y.value)

* (generic function with 350 methods)

In [ ]:
*(f::Dual, g::Dual) = Dual(f.value * g.value,
                         f.value * g.deriv + f.deriv * g.value)

In [44]:
value(f::Dual) = f.value
deriv(f::Dual) = f.deriv

deriv (generic function with 1 method)

In [45]:
*(f::Dual, g::Dual) = Dual(value(f) * value(g),
                         value(f) * deriv(g) + deriv(f) * value(g))

* (generic function with 350 methods)

In [46]:
∂(f::Dual) = f.deriv

∂ (generic function with 1 method)

In [47]:
*(f::Dual, g::Dual) = Dual(value(f) * value(g),
                         value(f) * ∂(g) + ∂(f) * value(g))

* (generic function with 350 methods)

(Try using `@code_llvm`)

In [38]:
f = Dual(3.0, 4.0)
g = Dual(5.0, 6.0)

Dual(5.0, 6.0)

In [39]:
f, g

(Dual(3.0, 4.0), Dual(5.0, 6.0))

In [40]:
f + g

Dual(8.0, 10.0)

In [41]:
f * g

Dual(15.0, 38.0)

Amazingly, we now have enough to be able to differentiate simple Julia functions involving only `+` and `*`. Define

In [43]:
a = 3.0
xx = Dual(a, 1.0)  # "the identity function x ↦ x, with derivative 1"

Dual(3.0, 1.0)

We initialize the derivative as 1.0 when we make a `Dual`. If we use `x` then we automatically differentiate!

#### Exercise 3

1. Define `a = 3.0` and `xx = Dual(a, 1.0)`.

    (i) Compute `xx + xx`. The result should have the value $2a$ and the derivative $2$ -- write a test that it does so.
    
    (ii) Do the same for `xx * xx`. 
    
    
2. Define the function `f(x) = x * x + x`. Compute `f(xx)` and check that it gives the correct value and derivative!


3. Does this work for the function `f(x) = x^2 + x`?  What do you need to do?


4. What happens for `f(x) = x^2 + 2x`? What do you need to do?


5. What should you do for `f(x) = sin(x) + x`?

In [48]:
a = 3.0
xx = Dual(a, 1.0)

Dual(3.0, 1.0)

In [49]:
xx + xx

Dual(6.0, 2.0)

In [50]:
xx * xx

Dual(9.0, 6.0)

In [51]:
xx * xx + xx  # the function x ↦ x^2 + x,  derivative 2x + 1

Dual(12.0, 7.0)

In [54]:
ff(x) = x*x*x + x*x + x*x  # x^3 + 2x^2

ff (generic function with 1 method)

In [55]:
ff(xx)

Dual(45.0, 39.0)

In [57]:
ff(a)

45.0

In [58]:
dff(x) = 3x^2 + 4x

dff (generic function with 1 method)

In [59]:
dff(a)

39.0

In [61]:
fff(x) = x^2

fff (generic function with 1 method)

In [62]:
fff(xx)

MethodError: MethodError: no method matching ^(::Dual, ::Int64)
Closest candidates are:
  ^(!Matched::Float16, ::Integer) at math.jl:795
  ^(!Matched::Missing, ::Integer) at missing.jl:124
  ^(!Matched::Missing, ::Number) at missing.jl:97
  ...

In [63]:
import Base: ^

In [64]:
^(x::Dual, n::Integer) = Base.power_by_squaring(x, n)

^ (generic function with 65 methods)

In [67]:
n = 4
fff(x) = x^n

fff (generic function with 1 method)

In [68]:
fff(xx)

Dual(81.0, 108.0)

In [70]:
fff(x) = x^4

fff (generic function with 1 method)

In [73]:
fff(xx)

Dual(81.0, 108.0)

In [ ]:
x::Dual ^ n::Integer = ..

`Base.literal_pow`

#### Exercise 4

1. Define a function `differentiate` that differentiates a function `f` at a point `a` using `Dual` numbers, by following the above pattern. (It should return just the derivative at the given point.)

This is the basis of ("forward-mode") automatic differentiation. The `ForwardDiff.jl` method contains a sophisticated implementation of this method.

### Parametric types

For simplicity, in the above we fixed the fields in the `Dual` type to be of type `Float64`. By doing so we are actually *losing power*. Instead we should let Julia "fill in" the types. 

To do so, we specify that we want to use a **type parameter** `T`. We can think of this as a "special kind of variable" that can only take on certain kinds of values. We specify this with the following syntax: 

In [80]:
struct MyType2{T}
    a::T
    b::T
end

[Note that we have not reused the name `MyType` since Julia *does not allow types to be redefined in a different way*.]

Here we are specifying that both fields `a` and `b` must share the same type `T`, but we have not restricted what values `T` can take. When we create an object, Julia will *infer* (work out) the type:

In [81]:
x = MyType2(3, 4)

MyType2{Int64}(3, 4)

In [16]:
y = MyType2(3.1, 4.2)

MyType2{Float64}(3.1, 4.2)

In [82]:
z = MyType2(1, 5.3)

MethodError: MethodError: no method matching MyType2(::Int64, ::Float64)
Closest candidates are:
  MyType2(::T, !Matched::T) where T at In[80]:2

In [83]:
struct MyType3{S,T}
    a::S
    b::T
end

In [85]:
x = MyType3(1, 5.3)

MyType3{Int64,Float64}(1, 5.3)

Note that `x` and `y` have *different* types.

We can define functions acting on parametric types without necessarily talking about the type parameter:

#### Exercise 5

1. Define a function that takes an object of type `MyType2`, *without* mentioning the type parameter, and returns the sum of the two fields.

   What happens when you apply this function to `x` and `y`?
   
   
2. Define a type `Dual2` with a type parameter `T` and the same functions `+` and `*` as before.


3. Define the function `f(x) = x * x + x`. What happens if you pass in `Dual` numbers with different type parameters?

In [75]:
struct DualReal
    value::Real
    deriv::Real
end
    

In [76]:
DualReal(big(1.0), big"1.4")

DualReal(1.0, 1.399999999999999999999999999999999999999999999999999999999999999999999999999997)

In [77]:
DualReal(3.1, big"1.4")

DualReal(3.1, 1.399999999999999999999999999999999999999999999999999999999999999999999999999997)

In [78]:
struct DualUntyped
    value
    deriv
end
    

In [79]:
DualUntyped("hello", "David")

DualUntyped("hello", "David")

In [ ]:
struct DualBig
    value::BigFloat
    deriv::BigFloat
end
    

In [86]:
struct Dual3{T<:Real}
    a::T
    b::T
end

In [87]:
Dual3("a", "b")

MethodError: MethodError: no method matching Dual3(::String, ::String)

In [88]:
Dual3(big"1.0", big"1.4")

Dual3{BigFloat}(1.0, 1.399999999999999999999999999999999999999999999999999999999999999999999999999997)

In [89]:
Dual3(1, 3.1)

MethodError: MethodError: no method matching Dual3(::Int64, ::Float64)
Closest candidates are:
  Dual3(::T<:Real, !Matched::T<:Real) where T<:Real at In[86]:2

In [90]:
methods(Dual3)

# 1 method for generic function "(::Type)":
[1] (::Type{Dual3})(a::T, b::T) where T<:Real in Main at In[86]:2

In [95]:
Dual3(a::Real, b::Real) = Dual3(promote(a, b))

Dual3

In [98]:
Dual3(a::Real, b::Real) = Dual3(promote(a, b)...)

Dual3

In [99]:
Dual3(3, 4.5)

Dual3{Float64}(3.0, 4.5)

In [96]:
promote(1, 3.4)

(1.0, 3.4)

In [94]:
@which promote_type(Int, Float64)

promote_type(::Type{T}, ::Type{S}) where {T, S} in Base at promotion.jl:215

In [100]:
Complex(3, 4.5)

3.0 + 4.5im

In [101]:
@which Complex(3, 4.5)

(::Type{Complex})(x::Real, y::Real) in Base at complex.jl:15

In [102]:
x = Dual(3, 4)

Dual(3.0, 4.0)

In [103]:
x isa Number

false

In [104]:
subtypes(Real)

4-element Array{Any,1}:
 AbstractFloat     
 AbstractIrrational
 Integer           
 Rational          

In [105]:
subtypes(AbstractFloat)

4-element Array{Any,1}:
 BigFloat
 Float16 
 Float32 
 Float64 

In [106]:
subtypes(Float64)

0-element Array{Type,1}

In [107]:
isconcretetype(Float64)

true

In [109]:
x = 3.1

3.1

In [110]:
typeof(x)

Float64

In [111]:
x isa AbstractFloat

true

In [112]:
gg(x::AbstractFloat) = sqrt(x)

gg (generic function with 1 method)

In [113]:
gg(π)

MethodError: MethodError: no method matching gg(::Irrational{:π})
Closest candidates are:
  gg(!Matched::AbstractFloat) at In[112]:1

In [ ]:
gg(x::T) where {T <: AbstractFloat} = sqrt(x)

In [115]:
struct Hello{T}
    a::T
    b::T
    c::T
end

In [ ]:
add(x::Hello, y::Hello) = x.a + y.a, x.b + y.b, ...

In [117]:
x = Hello(3, 4, 45)

Hello{Int64}(3, 4, 45)

In [121]:
myfields = fieldnames(typeof(x))

(:a, :b, :c)

In [120]:
getfield(x, :a)

3

In [123]:
x

Hello{Int64}(3, 4, 45)

In [125]:
[getfield(x, name) for name in myfields]

3-element Array{Int64,1}:
  3
  4
 45

In [126]:
abstract type AbstractDual{S<:Real,T<:Real}

struct Dual5{S<:Real, T<:Real} <: AbstractDual{S,T}
    a::S
    b::T
end

In [128]:
getfield.(myfields)

ArgumentError: ArgumentError: getfield: too few arguments (expected 2)